In [1]:
!python3 --version

Python 3.10.12


#### fix for could not import -lmza
https://github.com/lucidrains/imagen-pytorch/issues/92 

#### Fix for Can't import datasets AttributeError: partially initialized module 'datasets' has no attribute 'utils' (most likely due to a circular import)

Restart juypyter and kernel


In [2]:
import warnings
warnings.filterwarnings('ignore')


In [3]:
import helper
import time
helper.check_gpu_capacity()

GPU Information:
GPU 0:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB
GPU 1:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB
GPU 2:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB
GPU 3:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB


In [4]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

print(f"dataset size: {len(dataset)}")

# dataset size: 15011


Found cached dataset json (/home/ubuntu/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


dataset size: 15011


In [5]:
sample_ds = dataset[randrange(len(dataset))]

for key, value in sample_ds.items():
    print(f"--> {key}")
    print("====================")
    print(value)
    print("====================")

--> instruction
When is the World Teacher's Day celebrated?
--> context

--> response
World Teacher's Day is celebrated on 5th October
--> category
open_qa


In [6]:
set(dataset['category'])

{'brainstorming',
 'classification',
 'closed_qa',
 'creative_writing',
 'general_qa',
 'information_extraction',
 'open_qa',
 'summarization'}

In [7]:
# len(dataset.map( ['context']))

In [8]:
# list(filter(lambda x :x != '', list(map(lambda x:((x['category'], len(x['context'])) if len(x['context']) > 5000 else ''), dataset))))

Based on our use case, we want to focus on 'general_qa' and  'information_extraction tasks.,

In [9]:
target_categories = ['closed_qa', 'summarization', 'information_extraction']
filtered_dataset = dataset.filter(lambda x: x['category'] in target_categories)

print(set(filtered_dataset['category']))
filtered_dataset

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-67fe253a0190166e.arrow


{'summarization', 'closed_qa', 'information_extraction'}


Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 4467
})

In [10]:
sample_ds = filtered_dataset[randrange(len(filtered_dataset))]

for key, value in sample_ds.items():
    print(f"--> {key}")
    print("====================")
    print(value)
    print("====================")

--> instruction
Given this paragraph, what key crops were not available in Europe before the colonization of the New World?
--> context
Food historian Lois Ellen Frank calls potatoes, tomatoes, corn, beans, squash, chili, cacao, and vanilla the "magic eight" ingredients that were found and used only in the Americas before 1492 and were taken via the Columbian Exchange back to the Old World, dramatically transforming the cuisine there. According to Frank,
If we deconstruct that these foods were inherently native, then that means that the Italians didn't have the tomato, the Irish didn't have the potato, half the British National Dish—Fish and Chips—didn't exist. The Russians didn't have the potato, nor did they have vodka from the potato. There were no chiles in any Asian cuisine anywhere in the world, nor were there any chiles in any East Indian cuisine dishes, including curries. And the French had no confection using either vanilla or chocolate. So the Old World was a completely diffe

To instruct tune our model, we need to convert our structured examples into a collection of tasks described via instructions. We define a formatting_function that takes a sample and returns a string with our format instruction.

In [11]:
# def format_instruction(sample):
#     return f"""### Instruction:
#             Use the Input and context below to create an instruction, which could have been used to generate the input using an LLM. 
            
#             ### Input:
#             {sample['response']}
    
#             ### Context:
#             {sample['context'] if sample['context'] != '' else ''}
    
#             ### Response:
#             {sample['instruction']}
#         """

def format_instruction(sample):
    return f"""### Instruction:
            Use the Input and context below to create an instruction, which could have been used to generate the input using an LLM.             
            {sample['context'] if sample['context'] != '' else ''}
            
            {sample['response']}

            ### Response:
            {sample['instruction']}
        """


Let's test our formatting function on a random example.

In [12]:
from random import randrange

print(format_instruction(filtered_dataset[randrange(len(filtered_dataset))]))

## Do I need to add the context in the format_instruction?????

### Instruction:
            Use the Input and context below to create an instruction, which could have been used to generate the input using an LLM.             
            Long before any knowledge of electricity existed, people were aware of shocks from electric fish. Ancient Egyptian texts dating from 2750 BCE referred to these fish as the "Thunderer of the Nile", and described them as the "protectors" of all other fish. Electric fish were again reported millennia later by ancient Greek, Roman and Arabic naturalists and physicians. Several ancient writers, such as Pliny the Elder and Scribonius Largus, attested to the numbing effect of electric shocks delivered by electric catfish and electric rays, and knew that such shocks could travel along conducting objects. Patients with ailments such as gout or headache were directed to touch electric fish in the hope that the powerful jolt might cure them.
Ancient cultures around the Mediterranean knew that certain objects, such as rods of

In [13]:
print(format_instruction(sample_ds))

### Instruction:
            Use the Input and context below to create an instruction, which could have been used to generate the input using an LLM.             
            Food historian Lois Ellen Frank calls potatoes, tomatoes, corn, beans, squash, chili, cacao, and vanilla the "magic eight" ingredients that were found and used only in the Americas before 1492 and were taken via the Columbian Exchange back to the Old World, dramatically transforming the cuisine there. According to Frank,
If we deconstruct that these foods were inherently native, then that means that the Italians didn't have the tomato, the Irish didn't have the potato, half the British National Dish—Fish and Chips—didn't exist. The Russians didn't have the potato, nor did they have vodka from the potato. There were no chiles in any Asian cuisine anywhere in the world, nor were there any chiles in any East Indian cuisine dishes, including curries. And the French had no confection using either vanilla or chocolate. 

In [14]:
shuffled_dataset = filtered_dataset.shuffle(seed=42)
shuffled_dataset = shuffled_dataset.train_test_split(test_size=0.1)

train_dataset = shuffled_dataset['train']
test_dataset = shuffled_dataset['test']

train_eval_dataset = train_dataset.train_test_split(test_size=0.2)
train_dataset = train_eval_dataset['train']
eval_dataset = train_eval_dataset['test']


# Print the sizes of the train, eval, andtest sets
print("Dataset size:", len(dataset))
print("Train size:", len(train_dataset))
print("Eval size:", len(eval_dataset))
print("Test size:", len(test_dataset))

Loading cached shuffled indices for dataset at /home/ubuntu/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-be8fe28074092229.arrow


Dataset size: 15011
Train size: 3216
Eval size: 804
Test size: 447


### Instruction-tune Llama 2 using trl and the SFTTrainer

In [15]:
!pip install optimum -q

In [16]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

use_flash_attention = False
helper.check_gpu_capacity()


GPU Information:
GPU 0:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB
GPU 1:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB
GPU 2:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB
GPU 3:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB


In [17]:
# Hugging Face model id
model_llama_2_7b_id = "NousResearch/Llama-2-7b-hf" # non-gated
# model_id = "meta-llama/Llama-2-7b-hf" # gated

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
# torch.cuda.set_device(1)
# Load model and tokenizer
model_llama_2_7b = AutoModelForCausalLM.from_pretrained(model_llama_2_7b_id, 
                                                        quantization_config=bnb_config, 
                                                        use_cache=False, 
                                                        device_map="auto")
# model_llama_2_7b.to_bettertransformer()
tokenizer_llama_2_7b = AutoTokenizer.from_pretrained(model_llama_2_7b_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# sample_ds = filtered_dataset[35]

sample_ds = filtered_dataset[randrange(len(filtered_dataset))]

# prompt = f"""### Instruction:
#             Use the Input and context below to create an instruction, which could have been used to generate the input using an LLM. 
            
#             ### Input:
#             {sample_ds['response']}

#             ### Context:
#             {sample_ds['context'] if sample_ds['context'] != '' else ''}

#             ### Response:
#         """

prompt = f"""### Instruction:
            Use the Input below to create an instruction, which could have been used to generate the input using an LLM.             
            {sample_ds['context']}
            
            {sample_ds['response']}

            ### Response:
        """

input_ids = tokenizer_llama_2_7b(prompt, return_tensors="pt", 
                                 # max_length=2000,
                                 truncation=True
                                ).input_ids.to('cuda')
# with torch.inference_mode():
outputs = model_llama_2_7b.generate(input_ids=input_ids, max_new_tokens=200, 
                                    do_sample=True, top_p=0.9, temperature=0.9
                                   )

print(f"Prompt:\n{prompt}\n")
print(f"Generated instruction:\n{tokenizer_llama_2_7b.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"Ground truth:\n{sample_ds['instruction']}")

In [19]:
## From above, we see that the generated response is not okay

In [20]:

model_llama_2_7b.config.pretraining_tp = 1

# tokenizer_llama_2_7b = AutoTokenizer.from_pretrained(model_llama_2_7b_id)
tokenizer_llama_2_7b.pad_token = tokenizer_llama_2_7b.eos_token
tokenizer_llama_2_7b.padding_side = "right"

The SFTTrainer supports a native integration with peft, which makes it super easy to efficiently instruction tune LLMs. We only need to create our LoRAConfig and provide it to the trainer.

In [21]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"



In [22]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)

# prepare model for training
model_llama_2_7b = prepare_model_for_kbit_training(model_llama_2_7b)
model_llama_2_7b = get_peft_model(model_llama_2_7b, peft_config)
# print(print_number_of_trainable_model_parameters(model_llama_2_7b_peft))
model_llama_2_7b.print_trainable_parameters()

trainable params: 33,554,432 || all params: 3,533,967,360 || trainable%: 0.9494833591219133


Before we can start our training we need to define the hyperparameters (TrainingArguments) we want to use.

In [23]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="models/peft-llama-7-int4-instruct-generation-training",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    # save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=False, # disable tqdm since with packing values are in correct
    save_total_limit = 2,
    save_strategy="no",
    load_best_model_at_end=True
)


In [24]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='NousResearch/Llama-2-7b-hf', revision=None, task_type='CAUSAL_LM', inference_mode=False, r=64, target_modules=['q_proj', 'v_proj'], lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)

In [25]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

max_seq_length = 2048 # max sequence length for model and packing of the dataset

# response_template = " ### Response:"
# collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer_llama_2_7b)

trainer = SFTTrainer(
    model=model_llama_2_7b,
    train_dataset=train_dataset,
    # eval_dataset=eval_dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer_llama_2_7b,
    packing=True,
    formatting_func=format_instruction,
    args=args,
    # data_collator=collator,

)


In [26]:
helper.check_gpu_capacity()
# del model_llama_2_7b
# del model_llama_2_7b_kbit
helper.free_gpu_memory()
helper.memory_stats()


GPU Information:
GPU 0:
  Total Memory: 23028 MB
  Free Memory: 20511 MB
  Used Memory: 2004 MB
GPU 1:
  Total Memory: 23028 MB
  Free Memory: 20813 MB
  Used Memory: 1702 MB
GPU 2:
  Total Memory: 23028 MB
  Free Memory: 20813 MB
  Used Memory: 1702 MB
GPU 3:
  Total Memory: 23028 MB
  Free Memory: 20511 MB
  Used Memory: 2004 MB
GPU Memory Used: 1.12 GB
GPU Memory Cached: 1.20 GB


In [ ]:
%%time

# train
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model
trainer.save_model()
# After training, access the path of the best checkpoint like this
best_ckpt_path = trainer.state.best_model_checkpoint

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,1.521900


In [ ]:
args.output_dir

## Test Model and run Inference

In [27]:
if use_flash_attention:
    # unpatch flash attention
    from utils.llama_patch import unplace_flash_attn_with_attn
    unplace_flash_attn_with_attn()

import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

# args.output_dir = "llama-7-int4-dolly-filtered"

# load base LLM model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(args.output_dir)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Let’s load the dataset again with a random sample to try to generate an instruction.

In [146]:
# from datasets import load_dataset
# from random import randrange


# # Load dataset from the hub and get a sample
# dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
# sample = dataset[randrange(len(dataset))]

# prompt = f"""### Instruction:
# Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

# ### Input:
# {sample_ds['response']}

# ### Response:
# """

prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
4.1.	UV-Light
Together with oxygen, UV-light can produce a singlet oxygen which can form directly a hydroperoxide from a unsaturated lipid (see Figure 3). Usually during the processing of powders, prevention of UV-light intrusion is being controlled. Furthermore, it depends on the package used, but as well how the consumer stores the product. 
4.2.	Oxygen
Reaction with oxygen can take at various sites in the lipid oxidation chain reaction (see Figure 3). Exclusion of oxygen means mainly flushing inert gases typically nitrogen (or argon) to displace oxygen, packaging in vacuum or in modified atmosphere packaging (MAP). Oxygen scavengers can be employed to completely remove residual oxygen from the system. 
4.3.	Heat
Higher temperatures speed up the lipid oxidation reaction rates. Although the product should be prevented from heat, during production heat is needed to: 
1.	be able to handle ingredients (e.g. melting of fat, best done under nitrogen)
2.	get a product microbiologically stable
3.	spray dry the product
Clearly heat is needed during production but where possible it should be minimized to prevent lipid oxidation. 

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.2)

# print(f"Prompt:\n{sample_ds['response']}\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
# print(f"Ground truth:\n{sample_ds['instruction']}")


Generated instruction:
What are the main factors that influence lipid oxidation in food?
        


In [204]:
sample_ds = filtered_dataset[randrange(len(filtered_dataset))]

prompt = f"""### Instruction:
            Use the Input below to create an instruction, which could have been used to generate the input using an LLM. 
       
            {sample_ds['response']}
            {sample_ds['context'] if sample_ds['context'] != '' else ''}

        """

input_ids = tokenizer(prompt, return_tensors="pt", 
                                 # max_length=4000,
                                 truncation=True
                                ).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=200, 
                        do_sample=True, 
                         top_p=0.8, 
                         temperature=0.9
                                   )

# print(f"Prompt:\n{prompt}\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"Ground truth:\n{sample_ds['instruction']}")

Generated instruction:

            ### Context:
            This is a list of sovereign states and dependent territories in Africa. It includes both fully recognised states, states with limited or zero recognition, and dependent territories of both African and non-African states. It lists 56 sovereign states (54 of which are member states of the United Nations), two non-sovereign (dependent) territories of non-African sovereign states, and ten sub-national regions of non-African sovereign states. Malta and parts of France, Italy, Portugal, and Spain are located on the African continental plate, some considerably closer to the African mainland than the European mainland but, politically, are generally considered to be European by convention. Egypt, although extending into Asia through the Sinai Peninsula, is considered an African state.
    
            ### Response:
            How many sovereign
Ground truth:
What is the total number of geographical entities (states, territories, reg

### Evaluate the Model Quantitatively (with ROUGE Metric)
Perform inferences for the sample of the test dataset (only 50 instructions and responses to save time). 

In [96]:
dataset

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 15011
})

In [108]:
for key, value in test_dataset[50:52].items():
    print(key)
    print(value)

instruction
["Give me a bulleted list of Aaron Fenster's accomplishments.", 'Did Thomas Attewell bat with a right or left hand?']
context
['Aaron Fenster is a medical physicist at the University of Western Ontario Robarts Research Institute in London, Ontario, Canada. He was named a Fellow of the Institute of Electrical and Electronics Engineers (IEEE) in 2013 for his contributions to medical imaging and three-dimensional ultrasound-guided interventions. He is also a fellow of the Canadian Academy of Health Sciences and co-program director of the Ontario Institute for Cancer Research Imaging Program. He holds Ph.D. from the University of Toronto and received further training at the Ontario Cancer Institute.', 'Thomas Attewell (7 November 1869 – 6 July 1937) was an English first-class cricketer. Attewell was a right-handed batsman who bowled right-arm medium pace. He was born at Keyworth, Nottinghamshire.\n\nHis brother William played first-class cricket for Nottinghamshire and the Mary

In [205]:
import pandas as pd


instruction_test = test_dataset['instruction'][50:100]
response_test = test_dataset['response'][50:100]
context_test = test_dataset['context'][50:100]


generated_instruction_list = []
ground_truth_list = []

for instruction, response, context in zip(instruction_test, response_test, context_test):
    prompt = f"""### Instruction:
            Use the Input below to create an instruction, which could have been used to generate the input using an LLM. 
            
            {response}
            {context if context != '' else ''}

        """

    input_ids = tokenizer(prompt, return_tensors="pt", 
                                     # max_length=4000,
                                     truncation=True
                                    ).input_ids.cuda()
    # with torch.inference_mode():
    outputs = model.generate(input_ids=input_ids, max_new_tokens=200, 
                            do_sample=True, 
                             top_p=0.8, 
                             temperature=0.9
                                       )
    
    # print(f"Prompt:\n{sample_ds['response']}\n")
    generated_instruction = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]
    # generated_instruction = generated_instruction.split('\n')[0].strip()

    generated_instruction_list.append(generated_instruction)
    ground_truth_list.append(instruction)

zipped_summaries = list(zip(generated_instruction_list, ground_truth_list))
 
df = pd.DataFrame(zipped_summaries, columns = ['generated_instruction_list', 'ground_truth_list'])
df

,generated_instruction_list,ground_truth_list
0,\n ### Response:\n What ...,Give me a bulleted list of Aaron Fenster's acc...
1,\n ### Input:\n His brot...,Did Thomas Attewell bat with a right or left h...
2,\n ### Context:\n Honolu...,What is the capital of Hawaii?
3,\n ### Input:\n An AI sy...,What is the main prerequisite for an AI to sur...
4,\n ### Context:\n Artifi...,What are artificial neural networks?
5,"\n ### Context:\n ""Colou...",What is Color of the world
6,"\n ### Input:\n Eggs, ha...",Extract the ingredients needed to make pasta c...
7,\n ### Response:\n What ...,Who was the best team in the NFL during the 19...
8,\n ### Response:\n Who w...,Who was the first African American to go to sp...
9,"\n ### Input:\n Haakon, ...",Give me a quick brief of the Prince of Norway


In [210]:
for index, row in df[0:50].iterrows():
    print('GI: ', row['generated_instruction_list'])

    print('GT: ',row['ground_truth_list'], '\n')
    print('===============')

GI:  
            ### Response:
            What are some notable achievements of Aaron Fenster?
        
GT:  Give me a bulleted list of Aaron Fenster's accomplishments. 

GI:  
            ### Input:
            His brother William played first-class cricket for Nottinghamshire and the Marylebone Cricket Club, as well as Test cricket for England.
    
            ### Context:
            William Attewell (25 May 1863 – 25 May 1932) was an English first-class cricketer. Attewell was a right-handed batsman who bowled right-arm medium pace. He was born at Keyworth, Nottinghamshire.

His brother Thomas played first-class cricket for Nottinghamshire and the Marylebone Cricket Club, as well as Test cricket for England. His cousin Walter Attewell played first-class cricket for Nottinghamshire.
    
            ### Response:
            Who played Test cricket for England?
        
GT:  Did Thomas Attewell bat with a right or left hand? 

GI:  
            ### Context:
            Honolulu (

In [207]:
!pip install evaluate==0.4.0 rouge_score==0.1.2 --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Compute ROUGE score for this subset of the data. 

In [208]:
import evaluate

rouge = evaluate.load('rouge')

model_results = rouge.compute(
    predictions=generated_instruction_list,
    references=ground_truth_list[0:len(generated_instruction_list)],
    use_aggregator=True,
    use_stemmer=True,
)

print('MODEL:')
model_results_df = pd.DataFrame(model_results.items(), columns=['Metric', 'Value'])
model_results_df

MODEL:


,Metric,Value
0,rouge1,0.261189
1,rouge2,0.148227
2,rougeL,0.242753
3,rougeLsum,0.244149
